In [33]:
from binance.spot import Spot
client = Spot(base_url ='https://api2.binance.com')
#cap_client = Spot(endpoint='https://www.binance.com/exchange-api/v2/public/asset-service/product/get-products')
import pandas as pd
from datetime import datetime as dt
from math import ceil
import os

In [4]:
%cd G:\Meine Ablage\TechLabs\Scam Busters

G:\Meine Ablage\TechLabs\Scam Busters


In [4]:
data_root=r'data\03_label_dataset'
files = os.listdir(data_root)

In [5]:
#%cd ..

hier anpassen



In [6]:
pumps_df = pd.read_csv(data_root+'/'+files[0])

In [7]:
# pumps_df=pd.DataFrame([{
#     'exchange':'binance',
#     'datetime':'2020-10-18 18:00:18+00:00',
#     'coin':'oax'
# }])

In [8]:
binance_pumps = pumps_df.loc[pumps_df.exchange =='binance'].copy()
binance_pumps.loc[:,'datetime'] = pd.to_datetime([
    dtstring[:-6]
    for dtstring in binance_pumps.datetime
])
from datetime import timezone
binance_pumps.loc[:,'tz_utc'] = [
    dt_value.replace(tzinfo=timezone.utc).timestamp()
    for dt_value in binance_pumps.datetime
]

In [9]:
# set input parameters
# pairing_list = [INPT]
end_date = dt.now()
timeframe_d = 365

In [58]:
# col names
cols = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'n_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

In [11]:
def get_endtimes(range_d, final_datetime):
    calls_n = ceil(range_d*24*60/1000)
    # convert final datetime to unix timestamp
    final_datetime= final_datetime.replace(second=0, microsecond=0)
    final_ts = int(dt.timestamp(final_datetime)*1000)
    # create anchor values
    anchors = range(-calls_n,1)
    # turn anchor values into timestamps for the api call
    call_times = [
        60*1000*1000*anchor + final_ts
        for anchor in anchors
    ]
    return call_times

In [21]:
def klines_call(pairing,end):
    try:
        data=client.klines(symbol=pairing,interval='1m',limit=1000,endTime=end)
        df = pd.DataFrame(data=data)
        return df
    except Exception as e:
        print(f'Error: {e}',pairing)

In [13]:
def get_pairing_data(pairing,range_d,final_datetime='now'):
    if final_datetime == 'now':
        end = dt.now()
    else:
        end = final_datetime
    endtimes_list = get_endtimes(range_d,end)
    df_list = [
        klines_call(pairing,time)
        for time in endtimes_list
    ]
    df = pd.concat(df_list)
    df.columns = cols
    df.drop(['ignore'],axis=1,inplace=True)
    df.loc[:,'pairing'] = pairing
    df.reset_index(inplace=True,drop=True)
    return df

In [14]:
def ohlc_value_converter(convstr):
    if convstr[:3] == '0.0':
        convstr = convstr[3:]
    return float(convstr)

def series_converter(ser):
    converted = [
        ohlc_value_converter(convstr)
        for convstr in ser
    ]
    return converted

def ohlc_converter(df):
    copy = df.copy()
    ohlc_list = ['open','high','low','close']
    for entry in ohlc_list:
        copy.loc[:,entry] = series_converter(copy.loc[:,entry])
    return copy

def non_ohlc_converter(df):
    copy = df.copy()
    copy.volume = df.volume.astype(float)
    copy.quote_asset_volume = df.quote_asset_volume.astype(float)
    copy.taker_buy_base_asset_volume = df.taker_buy_base_asset_volume.astype(float)
    copy.taker_buy_quote_asset_volume = df.taker_buy_quote_asset_volume.astype(float)
    return copy

def full_converter(df):
    df = ohlc_converter(df)
    df = non_ohlc_converter(df)
    return df

In [26]:
from datetime import timedelta
from os.path import exists

def get_save_data(events):
    # iterate over binance_pumps dataframe
    for entry in events.index:
        # take the pairing
        pumped_coin = events.coin[entry]
        pairing = '{}BTC'.format(pumped_coin.upper())
        # df für ein Pairing wird erstellt
        pumptime = events.datetime[entry]

        # check if data has beed saved before
        save_path = 'data/04_raw_financial/pump_{coin}_{date}_binance.csv'.format(coin=pumped_coin,date=dt.strftime(pumptime,'%Y%m%d'))
        if exists(save_path):
            print('skipped: {coin}, {date}'.format(coin=pumped_coin,date=dt.strftime(pumptime,'%Y%m%d')))
            continue
        # if data is new, try to request it from binance
        try:
            df = get_pairing_data(
                pairing,
                3,
                pumptime + timedelta(seconds = 5400)
            )
            # df wird sinnvoll benannt abgespeichert
            df.to_csv('data/04_raw_financial/pump_{coin}_{date}_binance.csv'.format(coin=pumped_coin,date=dt.strftime(pumptime,'%Y%m%d')))

            # convert the dtypes
            df = full_converter(df)
            df.to_csv('data/04_raw_financial/pump_{coin}_{date}_binance__simplified.csv'.format(coin=pumped_coin,date=dt.strftime(pumptime,'%Y%m%d')))
        except:
            pass
    print('------------------','done')

In [ ]:
get_save_data(binance_pumps)

In [62]:
def get_save_context_data(pairing,time):
    try:
        df = get_pairing_data(
            pairing,
            3,
            time + timedelta(seconds = 5400)
        )
        pumped_coin = pairing[:-3]
        # df wird sinnvoll benannt abgespeichert
        df.to_csv('data/04_raw_financial/context_{coin}_{date}_binance.csv'.format(coin=pumped_coin,date=dt.strftime(time,'%Y%m%d')))

        # convert the dtypes
        df = full_converter(df)
        df.to_csv('data/04_raw_financial/context_{coin}_{date}_binance__simplified.csv'.format(coin=pumped_coin,date=dt.strftime(time,'%Y%m%d')))
    except:
        pass

In [ ]:
# pd.read_csv('data/04_raw_financial/pump_{coin}_{date}_binance__simplified.csv'.format(coin='oax',date='20181018')).high.plot()

In [17]:
from random import randint, choice
import datetime
seconds_offset = 75600+3600 # leads to 22:00 gmt
def get_random_pumptime():
    start = 17532 # jan 2018
    end = 19069 # march 2022
    randy = randint(start,end)
    evening_ts = randy*3600*24+seconds_offset
    return datetime.datetime.utcfromtimestamp(evening_ts)


In [5]:
# request random coin data to improve training set
from binance.client import Client

api_key = "hsr4OHNEtsIowVxhYxLs3g6Vr0e8qlRzZacYSjeTxMPrHP2fCxoSnkiyjzkqgGrU"
api_secret = "F5zU6VRC66M2NiEKvf3SQmmUeHJCS77NgibSZmjZjLpkAZcca9MnUsVUmyz3tSQ0"

client = Client(api_key, api_secret)
exchange_info = client.get_exchange_info()
markets_list = exchange_info['symbols']
btc_markets = [
    market['symbol'] for market in markets_list if 'BTC' in market['symbol']
]

import pickle
with open('references/binance_btc_markets.pkl','wb') as f:
    pickle.dump(btc_markets, f)

len(btc_markets)

427

In [43]:
'ABCBTC'[:-3]

'ABC'

In [68]:
'ABCBTC'.replace('BTC','')

'ABC'

In [53]:
# check if data is available
def check_if_new_data(pair,time):
    save_path = 'data/04_raw_financial/pump_{coin}_{date}_binance.csv'.format(coin=pair.replace('BTC',''),date=dt.strftime(time,'%Y%m%d'))
    return not exists(save_path)

In [ ]:
[
    get_save_context_data(choice(btc_markets),get_random_pumptime())
    for i in range(1000)
]